In [1]:
!pip install "transformers==4.31.0" "datasets[s3]==2.13.0" sagemaker --upgrade --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.29.3 requires PyYAML<5.5,>=3.10, but you have pyyaml 6.0.1 which is incompatible.
farm-haystack 1.15.0 requires transformers[torch]==4.25.1, but you have transformers 4.31.0 which is incompatible.


In [2]:
!huggingface-cli login --token YOUR_ACCESS_TOKEN

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\Tedd\.cache\huggingface\token
Login successful


In [2]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='AWSSagemakerExec')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")



Couldn't call 'get_role' to get Role ARN from role name tedd-de to get Role path.


sagemaker role arn: arn:aws:iam::442751047058:role/AWSSagemakerExec
sagemaker bucket: sagemaker-ap-south-1-442751047058
sagemaker session region: ap-south-1


In [3]:
from datasets import load_dataset
from random import randrange

# Load dataset from the hub
dataset = load_dataset("databricks/databricks-dolly-15k", split="train")

print(f"dataset size: {len(dataset)}")
print(dataset[randrange(len(dataset))])
# dataset size: 15011

c:\Users\Tedd\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 497.84it/s]
                                                                    

Dataset json downloaded and prepared to C:/Users/Tedd/.cache/huggingface/datasets/databricks___json/databricks--databricks-dolly-15k-7427aa6e57c34282/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.
dataset size: 15011
{'instruction': "Categorize these super heroes as 'Marvel' or 'DC Comics'", 'context': '', 'response': 'Thor, Flash, Iron Man, Super Man, Wonder Woman, Captain America', 'category': 'classification'}


In [4]:
def format_dolly(sample):
    instruction = f"### Instruction\n{sample['instruction']}"
    context = f"### Context\n{sample['context']}" if len(sample["context"]) > 0 else None
    response = f"### Answer\n{sample['response']}"
    # join all the parts together
    prompt = "\n\n".join([i for i in [instruction, context, response] if i is not None])
    return prompt

In [5]:
from random import randrange

print(format_dolly(dataset[randrange(len(dataset))]))


### Instruction
How do I decorate my new home?

### Answer
To decorate your new home, its good to get a strong sense of your own personal style and to seek inspiration from things you like. There are certain platforms like Pinterest where you can save images that you like to get inspiration from others, and see patterns in what type of decor speaks to you. Its good to invest in certain pieces that you will use a lot, like a mattress or couch, and then search for bargains on other pieces on secondhand marketplaces like Craigslist or Facebook marketplace. If you feel totally lost and overwhelmed, there are also interior design services available online to do remotely or you can hire a personal interior designer. Sometimes it is good to live in your space for a bit before making big purchases since your ideas on how to utilize the space may evolve as you spend more time in it.


In [6]:
from transformers import AutoTokenizer

model_id = "meta-llama/Llama-2-13b-hf" # sharded weights
tokenizer = AutoTokenizer.from_pretrained(model_id,use_auth_token=True)
tokenizer.pad_token = tokenizer.eos_token

OSError: meta-llama/Llama-2-13b-hf is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.